In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import os
import gzip
import tarfile
import gc
import torch.nn as nn
import torch.nn.functional as F
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

In [ ]:
class CellDetection(nn.Module):
    def __init__(self, dataset):
        super(CancerDetection, self).__init__()
        a = 3
        b = 64
        c = 128
        d = 256
        e = 512
        f = 1024
        self.conv1 = conv(a,b)
        self.conv2 = conv(b,c)
        self.conv3 = conv(c,d)
        self.conv4 = conv(d,e)
        self.conv5 = conv(e,f)
        self.conv6 = conv(f,e)
        self.conv7 = conv(e,d)
        self.conv8 = conv(d,c)
        self.conv9 = conv(c,b)
        self.conv10 = nn.Conv2d(64,2,1,1,0)
        self.trans1 = nn.ConvTranspose2d(1024,512,kernel_size = 2, stride = 2, padding = 0)
        self.trans2 = nn.ConvTranspose2d(512,256,kernel_size = 2, stride = 2, padding = 0)
        self.trans3 = nn.ConvTranspose2d(256,128,kernel_size = 2, stride = 2, padding = 0)
        self.trans4 = nn.ConvTranspose2d(128,64,kernel_size = 2, stride = 2, padding = 0)
        self.pool = nn.MaxPool2d(kernel_size = 2)
        
    def forward(self, x):
        conv1_out = self.conv1(x)
        conv2_out = self.conv2(self.pool(conv1_out))
        conv3_out = self.conv3(self.pool(conv2_out))
        conv4_out = self.conv4(self.pool(conv3_out)) 
        conv5_out = self.conv5(self.pool(conv4_out))
        conv6_out = self.conv6(torch.cat((self.trans1(conv5_out), conv4_out),1))
        conv7_out = self.conv7(torch.cat((self.trans2(conv4_out), conv3_out),1))
        conv8_out = self.conv8(torch.cat((self.trans3(conv3_out), conv2_out),1))
        conv9_out = self.conv9(torch.cat((self.trans4(conv2_out), conv1_out),1))
        conv10_out = self.conv10(conv9_out)
        return conv10_out
    
class conv(nn.Module):
    def __init__(self,a,b):
        super(conv, self).__init__()
        self.conv1 = nn.Conv2d(a,b,kernel_size = 3, stride = 1, padding = 1)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(b,b,kernel_size = 3, stride = 1, padding = 1)
        self.batch1 = nn.BatchNorm2d(b)
        self.batch2 = nn.BatchNorm2d(b)
        self.relu4 = nn.ReLU()
       
        
    def forward(self, x):
        conv1_out = self.conv1(x)
        batch1_out = self.batch1(conv1_out)
        relu2_out = self.relu2(batch1_out)
        conv3_out = self.conv3(relu2_out)
        batch2_out = self.batch2(conv3_out)
        relu4_out = self.relu4(batch2_out) 
        return relu4_out

def scope():
    try:
        #your code for calling dataset and dataloader
        train_dataset = pd.read_csv('Desktop/cell_detection/train.csv')
        val_dataset = pd.read_csv('Desktop/cell_detection/test.csv')
        final_test_img = val_dataset[172][0].cuda()
        final_test_label = val_dataset[172][1].cuda()
        model = CancerDetection(train_dataset)
        model = model.cuda()
        my_objective = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        train_loader = DataLoader(train_dataset, batch_size=2, pin_memory=True, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=2, pin_memory=True,shuffle=True)
        


        gc.collect()


        my_losses = []
        val_losses = []
        accuracy = []
        global accuracy_avg_loss
        accuracy_avg_loss = []
        global val_avg_loss 
        val_avg_loss = []
        global my_avg_loss 
        my_avg_loss = []
        loop = tqdm(total=len(train_loader)*6, position=0)
        counter = 0
        for epoch in range(6):

            for x, y_truth in train_loader:
                x, y_truth = x.cuda(async=True), y_truth.cuda(async=True) 

                optimizer.zero_grad()
                
                y_hat = model(x)
                
                my_loss = my_objective(y_hat, y_truth.long())

                my_losses.append(my_loss.item())
                accuracy.append((y_hat.argmax(1) == y_truth.long()).float().mean())

                loop.set_description('loss:{:.4f}'.format(my_loss.item()))
                loop.update(1)

                my_loss.backward()

                optimizer.step()
                
                if (counter % 100 == 0):
                    for x, y_truth in val_loader:
                        x, y_truth = x.cuda(async=True), y_truth.cuda(async=True)
                        
                        y_hat = model(x)
                        
                        loss = my_objective(y_hat, y_truth.long())
                        

                        val_losses.append(loss.item())
                counter+=1  
            val_avg_loss.append(np.mean(val_losses))
            val_losses.clear()
            my_avg_loss.append(np.mean(my_losses))
            my_losses.clear()
            accuracy_avg_loss.append(np.mean(accuracy))
            accuracy.clear()

        return(final_test_img, final_test_label, model(final_test_img.unsqueeze(0)))


        loop.close() 


    except:
        __ITB__()